<a href="https://colab.research.google.com/github/josedossantos10/HIRS/blob/main/Hibrido_evaluate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Ajuste os parâmentros baixo e depois execute todo o codigo para avaliar o modelo hibrido

In [ ]:
# Número de documentos a serem recuperados
top_k = 20
# Coluna a ser usada para avaliação na camada lexica
column_data_bm25 = 'txtInteiroTeor'
# Coluna a ser usada para avaliação na camada semântica
column_data_model = 'txtInteiroTeor'
# Caminho até o modelos semaântico (com Fine-Tuning) que será usado para recuperar os documentos
version = '/content/drive/MyDrive/Sentence Model Tuned/txtInteiroTeor_sbert'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install sentence_transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 5.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
#Importando a biblioteca nltk

import nltk
from sentence_transformers import SentenceTransformer, SentencesDataset, util, SentencesDataset, InputExample, losses
from tqdm import tqdm
import torch
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('rslp')
import pandas as pd

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Unzipping stemmers/rslp.zip.


## BM25

 ### *BM25* Okapi

In [ ]:
#Algoritmo do BM25 Okapi

import math
import numpy as np
from multiprocessing import Pool, cpu_count

"""
All of these algorithms have been taken from the paper:
Trotmam et al, Improvements to BM25 and Language Models Examined
Here we implement all the BM25 variations mentioned.
"""


class BM25:
    def __init__(self, corpus, tokenizer=None):
        self.corpus_size = len(corpus)
        self.avgdl = 0
        self.doc_freqs = []
        self.idf = {}
        self.doc_len = []
        self.tokenizer = tokenizer
        #Tokenizar o corpus
        if tokenizer:
            corpus = self._tokenize_corpus(corpus)

        nd = self._initialize(corpus)
        self._calc_idf(nd)

    def _initialize(self, corpus):
        nd = {}  # word -> number of documents with word
        num_doc = 0
        for document in corpus:
            self.doc_len.append(len(document))
            num_doc += len(document)

            frequencies = {}
            for word in document:
                if word not in frequencies:
                    frequencies[word] = 0
                frequencies[word] += 1
            self.doc_freqs.append(frequencies)

            for word, freq in frequencies.items():
                try:
                    nd[word] += 1
                except KeyError:
                    nd[word] = 1

        self.avgdl = num_doc / self.corpus_size

        return nd

    def _tokenize_corpus(self, corpus):
        pool = Pool(cpu_count())
        tokenized_corpus = pool.map(self.tokenizer, corpus)
        return tokenized_corpus

    def _calc_idf(self, nd):
        raise NotImplementedError()

    def get_scores(self, query):
        raise NotImplementedError()

    def get_batch_scores(self, query, doc_ids):
        raise NotImplementedError()

    def get_top_n(self, query, documents, n=5):

        assert self.corpus_size == len(documents), "The documents given don't match the index corpus!"

        scores = self.get_scores(query)
        top_n = np.argsort(scores)[::-1][:n]
        return [documents[i] for i in top_n]

    def get_top_n_ngram(self, score, documents, n=5):

        assert self.corpus_size == len(documents), "The documents given don't match the index corpus!"

        pdr = np.zeros(self.corpus_size)
        alpha = 2
        for ps in score:
            pdr = np.add(pdr, alpha * ps)
            alpha -= 1

        top_n = np.argsort(pdr)[::-1][:n]
        return [documents[i] for i in top_n]

    def get_partial_score(self, query, documents):
        assert self.corpus_size == len(documents), "The documents given don't match the index corpus!"

        score = self.get_scores(query)

        return score

#BM25Okapi
class BM25Okapi(BM25):
    def __init__(self, corpus, tokenizer=None, k1=1.5, b=0.75, epsilon=0.25):
        self.k1 = k1
        self.b = b
        self.epsilon = epsilon
        super().__init__(corpus, tokenizer)

    #Calculo do IDF (Inverse Document Frequency)
    def _calc_idf(self, nd):
        """
        Calculates frequencies of terms in documents and in corpus.
        This algorithm sets a floor on the idf values to eps * average_idf
        """
        # collect idf sum to calculate an average idf for epsilon value
        idf_sum = 0
        # collect words with negative idf to set them a special epsilon value.
        # idf can be negative if word is contained in more than half of documents
        negative_idfs = []
        for word, freq in nd.items():
            idf = math.log(self.corpus_size - freq + 0.5) - math.log(freq + 0.5)
            self.idf[word] = idf
            idf_sum += idf
            if idf < 0:
                negative_idfs.append(word)
        self.average_idf = idf_sum / len(self.idf)

        eps = self.epsilon * self.average_idf
        for word in negative_idfs:
            self.idf[word] = eps

    #Avaliar a pontuacao de todos os documentos na base
    def get_scores(self, query):
        """
        The ATIRE BM25 variant uses an idf function which uses a log(idf) score. To prevent negative idf scores,
        this algorithm also adds a floor to the idf value of epsilon.
        See [Trotman, A., X. Jia, M. Crane, Towards an Efficient and Effective Search Engine] for more info
        :param query:
        :return:
        """
        score = np.zeros(self.corpus_size)
        doc_len = np.array(self.doc_len)
        for q in query:
            q_freq = np.array([(doc.get(q) or 0) for doc in self.doc_freqs])
            score += (self.idf.get(q) or 0) * (q_freq * (self.k1 + 1) /
                                               (q_freq + self.k1 * (1 - self.b + self.b * doc_len / self.avgdl)))
        return score

    def get_batch_scores(self, query, doc_ids):
        """
        Calculate bm25 scores between query and subset of all docs
        """
        assert all(di < len(self.doc_freqs) for di in doc_ids)
        score = np.zeros(len(doc_ids))
        doc_len = np.array(self.doc_len)[doc_ids]
        for q in query:
            q_freq = np.array([(self.doc_freqs[di].get(q) or 0) for di in doc_ids])
            score += (self.idf.get(q) or 0) * (q_freq * (self.k1 + 1) /
                                               (q_freq + self.k1 * (1 - self.b + self.b * doc_len / self.avgdl)))
        return score.tolist()


### BM25L

In [ ]:
#Implementacao do BM25L - adapta parametros para corrigir a preferencia do Okapi por documentos mais curtos

class BM25L(BM25):
    def __init__(self, corpus, tokenizer=None, k1=1.5, b=0.75, epsilon=0.25):
        self.k1 = k1
        self.b = b
        self.epsilon = epsilon
        super().__init__(corpus, tokenizer)

    #Calculo do IDF (Inverse Document Frequency)
    def _calc_idf(self, nd):
        """
        Calculates frequencies of terms in documents and in corpus.
        This algorithm sets a floor on the idf values to eps * average_idf
        """
        # collect idf sum to calculate an average idf for epsilon value
        idf_sum = 0
        # collect words with negative idf to set them a special epsilon value.
        # idf can be negative if word is contained in more than half of documents
        negative_idfs = []
        for word, freq in nd.items():
            idf = math.log(self.corpus_size + 1) - math.log(freq + 0.5)
            self.idf[word] = idf
            idf_sum += idf
            if idf < 0:
                negative_idfs.append(word)
        self.average_idf = idf_sum / len(self.idf)

        eps = self.epsilon * self.average_idf
        for word in negative_idfs:
            self.idf[word] = eps

    #Calculo do ctd
    def get_ctd(self, q_freq, b, doc_len, avg_len):
      ctd = q_freq/(1 - b + b*(doc_len)/(avg_len))
      return ctd

    #Avaliar a pontuacao de todos os documentos na base
    def get_scores(self, query):
        """
        The ATIRE BM25 variant uses an idf function which uses a log(idf) score. To prevent negative idf scores,
        this algorithm also adds a floor to the idf value of epsilon.
        See [Trotman, A., X. Jia, M. Crane, Towards an Efficient and Effective Search Engine] for more info
        :param query:
        :return:
        """
        score = np.zeros(self.corpus_size)
        doc_len = np.array(self.doc_len)

        for q in query:
            q_freq = np.array([(doc.get(q) or 0) for doc in self.doc_freqs])
            ctd = self.get_ctd(q_freq, self.b, doc_len, self.avgdl)
            score += (self.idf.get(q) or 0) * ( (ctd + 0.5) * (self.k1 + 1) /
                                               ( (ctd + 0.5) + self.k1 ))
        return score

    def get_batch_scores(self, query, doc_ids):
        """
        Calculate bm25 scores between query and subset of all docs
        """
        assert all(di < len(self.doc_freqs) for di in doc_ids)
        score = np.zeros(len(doc_ids))
        doc_len = np.array(self.doc_len)[doc_ids]
        for q in query:
            q_freq = np.array([(self.doc_freqs[di].get(q) or 0) for di in doc_ids])
            score += (self.idf.get(q) or 0) * (q_freq * (self.k1 + 1) /
                                               (q_freq + self.k1 * (1 - self.b + self.b * doc_len / self.avgdl)))
        return score.tolist()

### BM25Plus

In [ ]:
#BM25+
class BM25plus(BM25):
    def __init__(self, corpus, tokenizer=None, k1=1.5, b=0.75, epsilon=0.25):
        self.k1 = k1
        self.b = b
        self.epsilon = epsilon
        super().__init__(corpus, tokenizer)

    #Calculo do IDF (Inverse Document Frequency)
    def _calc_idf(self, nd):
        """
        Calculates frequencies of terms in documents and in corpus.
        This algorithm sets a floor on the idf values to eps * average_idf
        """
        # collect idf sum to calculate an average idf for epsilon value
        idf_sum = 0
        # collect words with negative idf to set them a special epsilon value.
        # idf can be negative if word is contained in more than half of documents
        negative_idfs = []
        for word, freq in nd.items():
            idf = math.log(self.corpus_size + 1) - math.log(freq)
            self.idf[word] = idf
            idf_sum += idf
            if idf < 0:
                negative_idfs.append(word)
        self.average_idf = idf_sum / len(self.idf)

        eps = self.epsilon * self.average_idf
        for word in negative_idfs:
            self.idf[word] = eps

    #Avaliar pontuação de todos os documentos na base
    def get_scores(self, query):
        """
        The ATIRE BM25 variant uses an idf function which uses a log(idf) score. To prevent negative idf scores,
        this algorithm also adds a floor to the idf value of epsilon.
        See [Trotman, A., X. Jia, M. Crane, Towards an Efficient and Effective Search Engine] for more info
        :param query:
        :return:
        """
        score = np.zeros(self.corpus_size)
        doc_len = np.array(self.doc_len)

        for q in query:
            q_freq = np.array([(doc.get(q) or 0) for doc in self.doc_freqs])
            score += (self.idf.get(q) or 0) * ( (q_freq * (self.k1 + 1)) /
                                               (self.k1*((1-self.b) + self.b*((doc_len)/(self.avgdl)) + q_freq) + 1))
        return score


## Pré-processamento

### Savoy

In [ ]:

class Savoy:

    def __removeAllPTAccent(self, old_word):
        word = list(old_word)
        len_word = len(word)-1
        for i in range(len_word, -1, -1):
            if word[i] == 'ä':
                word[i] = 'a'
            if word[i] == 'â':
                word[i] = 'a'
            if word[i] == 'à':
                word[i] = 'a'
            if word[i] == 'á':
                word[i] = 'a'
            if word[i] == 'ã':
                word[i] = 'a'
            if word[i] == 'ê':
                word[i] = 'e'
            if word[i] == 'é':
                word[i] = 'e'
            if word[i] == 'è':
                word[i] = 'e'
            if word[i] == 'ë':
                word[i] = 'e'
            if word[i] == 'ï':
                word[i] = 'i'
            if word[i] == 'î':
                word[i] = 'i'
            if word[i] == 'ì':
                word[i] = 'i'
            if word[i] == 'í':
                word[i] = 'i'
            if word[i] == 'ü':
                word[i] = 'u'
            if word[i] == 'ú':
                word[i] = 'u'
            if word[i] == 'ù':
                word[i] = 'u'
            if word[i] == 'û':
                word[i] = 'u'
            if word[i] == 'ô':
                word[i] = 'o'
            if word[i] == 'ö':
                word[i] = 'o'
            if word[i] == 'ó':
                word[i] = 'o'
            if word[i] == 'ò':
                word[i] = 'o'
            if word[i] == 'õ':
                word[i] = 'o'
            if word[i] == 'ç':
                word[i] = 'c'

        new_word = "".join(word)
        return new_word

    def __finalVowelPortuguese(self, word):
        len_word = len(word)
        if len_word > 3:
            if word[-1] == 'e' or word[-1] == 'a' or word[-1] == 'o':
                word = word[:-1]

        return word

    def __remove_PTsuffix(self, word):
        len_word = len(word)

        if len_word > 3:
            if word[-1] == 's' and word[-2] == 'e' and (word[-3] == 'r' or word[-3] == 's' or word[-3] == 'z' or word[-3] == 'l'):
                word = word[:-2]
                return word
        if len_word > 2:
            if word[-1] == 's' and word[-2] == 'n':
                new_word = list(word)
                new_word[-2] = 'm'
                sing = "".join(new_word)
                sing = sing[:-1]
                return sing

        if len_word > 3:
            if (word[-1] == 's' and word[-2] == 'i') and (word[-3] == 'e' or word[-3] == 'é'):
                new_word = list(word)
                new_word[-3] = 'e'
                new_word[-2] = 'l'
                sing = "".join(new_word)
                sing = sing[:-1]
                return sing

        if len_word > 3:
            if word[-1] == 's' and word[-2] == 'i' and word[-3] == 'a':
                new_word = list(word)
                new_word[-2] = 'l'
                sing = "".join(new_word)
                sing = sing[:-1]
                return sing

        if len_word > 3:
            if word[-1] == 's' and word[-2] == 'i' and word[-3] == 'ó':
                new_word = list(word)
                new_word[-3] = 'o'
                new_word[-2] = 'l'
                sing = "".join(new_word)
                sing = sing[:-1]
                return sing

        if len_word > 3:
            if word[-1] == 's' and word[-2] == 'i':
                new_word = list(word)
                new_word[-1] = 'l'
                sing = "".join(new_word)
                return sing

        if len_word > 2:
            if word[-1] == 's' and word[-2] == 'e' and word[-3] == 'õ':
                new_word = list(word)
                new_word[-3] = 'ã'
                new_word[-2] = 'o'
                sing = "".join(new_word)
                sing = sing[:-1]
                return sing
            if word[-1] == 's' and word[-2] == 'e' and word[-3] == 'ã':
                new_word = list(word)
                new_word[-2] = 'o'
                sing = "".join(new_word)
                sing = sing[:-1]
                return sing

        if len_word > 5:
            if word[-1] == 'e' and word[-2] == 't' and word[-3] == 'n' and word[-4] == 'e' and word[-5] == 'm':
                word = word[:-5]
                return word

        if len_word > 2:
            if word[-1] == 's':
                word = word[:-1]

        return word

    def __normFemininPortuguese(self, word):

        len_word = len(word)

        if len_word < 3 or word[-1] != 'a':
            return word

        if len_word > 6:

            if word[-2] == 'h' and word[-3] == 'n' and word[-4] == 'i':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

            if word[-2] == 'c' and word[-3] == 'a' and word[-4] == 'i':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

            if word[-2] == 'r' and word[-3] == 'i' and word[-4] == 'e':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

        if len_word > 5:
            if word[-2] == 'n' and word[-3] == 'o':
                new_word = list(word)
                new_word[-3] = 'ã'
                new_word[-2] = 'o'
                masc = "".join(new_word)
                masc = masc[:-1]
                return masc

            if word[-2] == 'r' and word[-3] == 'o':
                word = word[:-1]
                return word

            if word[-2] == 's' and word[-3] == 'o':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

            if word[-2] == 's' and word[-3] == 'e':
                new_word = list(word)
                new_word[-3] = 'ê'
                masc = "".join(new_word)
                masc = masc[:-1]
                return masc

            if word[-2] == 'c' and word[-3] == 'i':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

            if word[-2] == 'd' and word[-3] == 'i':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

            if word[-2] == 'd' and word[-3] == 'a':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

            if word[-2] == 'v' and word[-3] == 'i':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

            if word[-2] == 'm' and word[-3] == 'a':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

            if word[-2] == 'n':
                new_word = list(word)
                new_word[-1] = 'o'
                masc = "".join(new_word)
                return masc

        return word

    def stem(self, word):
        len_word = len(word)
        if len_word > 2:
            word = self.__remove_PTsuffix(word)
            word = self.__normFemininPortuguese(word)
            word = self.__finalVowelPortuguese(word)
            word = self.__removeAllPTAccent(word)

        return word


### Aplicando pré-processamento

In [ ]:
from nltk.tokenize import word_tokenize
from string import punctuation
import nltk
from unicodedata import normalize
from nltk.stem import RSLPStemmer
from nltk.util import ngrams
from nltk.tokenize import RegexpTokenizer

# Remove os acentos de uma string
def _remove_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

# Tokenizador
def preprocess0(txt):
    terms = word_tokenize(txt)
    # tokenizer = RegexpTokenizer('\w+')
    # terms = tokenizer.tokenize(txt.lower())
 #   terms = [word for word in terms if word not in stopwords]

    return terms

# Remoção de stopwords + acentuação
def preprocess2(txt):
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))

    # terms = word_tokenize(txt.lower())
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [word for word in terms if word not in stopwords]

    return terms

# Remoção de stopwords + acentuação + steming
def preprocess3(txt):
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))

    # stemmer = RSLPStemmer()
    # terms = word_tokenize(txt.lower())
    stemmer = Savoy()
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [stemmer.stem(word) for word in terms if word not in stopwords]
    return terms

# Remoção de stopwords + acentuação + n-gram
def preprocess_ngram(txt, n):
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))

    # terms = word_tokenize(txt.lower())
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [word for word in terms if word not in stopwords]

    ngram = list(ngrams(terms, n))

    return ngram

# Remoção de stopwords + acentuação + steming + n-gram
def preprocess_ngram_stem(txt, n):
    txt = _remove_acentos(txt)
    stopwords = nltk.corpus.stopwords.words("portuguese")
    stopwords.extend(list(punctuation))

    # stemmer = RSLPStemmer()
    # terms = word_tokenize(txt.lower())
    stemmer = Savoy()
    tokenizer = RegexpTokenizer('\w+')
    terms = tokenizer.tokenize(txt.lower())
    terms = [stemmer.stem(word) for word in terms if word not in stopwords]

    ngram = list(ngrams(terms, n))

    return ngram

In [ ]:
# Obtem o nome do texto no df
def get_name(df, doc,column_data):
    return str(df[df[column_data]==doc].txtNome.to_numpy()[0]).strip()


In [ ]:
def evaluate_model(corpus_embeddings, query_embedding):
    # corpus_embeddings = model.encode(corpus, convert_to_tensor=True)
    res_vector = list()
    try:
        # query_embedding = model.encode(query, convert_to_tensor=True)
        cos_scores = util.pytorch_cos_sim(query_embedding, corpus_embeddings)[0]
        # top_results = torch.topk(cos_scores, k=top_k)

        # if torch.cuda.is_available():
        #     top_resuts_txt = [get_name(df, txt, in_field=column_data, out_field="txtNome").strip() for txt in corpus[top_results[1].cpu().numpy()]]
        # else:
        #     top_resuts_txt = [get_name(df, txt, in_field=column_data, out_field="txtNome").strip() for txt in corpus[top_results[1].numpy()]]

        # res_vector.append([get_name(df_assunto, query, "TxtAssunto", "NÚMERO-PROPOSIÇÃOSILEG").strip(), top_resuts_txt])
    except Exception as e:
        print(f"ERRO ao realizar encoding:{e}")
        pass
    if torch.cuda.is_available():
      return cos_scores.cpu().numpy()
    return cos_scores.numpy()

In [ ]:
# prompt: crie uma pasta data usando o pytohn

import os
if not os.path.exists('data'):
  os.makedirs('data')


In [ ]:
import requests
import re
def get_file(url, folder='data'):
  r = requests.get(url.split('?').pop(0)+'?download=1')
  d = r.headers['content-disposition']
  fname = re.findall("filename=(.+)", d)[0].replace('"','')
  with open(f'/content/{folder}/{fname}','wb') as f:
    f.write(r.content)
  if '.zip' in fname:
    !unzip -o '/content/{folder}/{fname}' -d '/content/{folder}/'
    print('Unziped and',end=' ')
  print('Saved '+fname)

In [ ]:
get_file('https://ufrpebr-my.sharepoint.com/:u:/g/personal/joseantonio_santos_ufrpe_br/EQcLlSaOKPtCtzY3juKPkpYB6Rl9Jnp-xNimWaIcJNe_bg?e=jc0ZWR')
get_file('https://ufrpebr-my.sharepoint.com/:x:/g/personal/joseantonio_santos_ufrpe_br/EfciCkFDXkxAoKnvwW-O0FYB1YMf-iXIwD1nVdf0Ve_t8g?e=X1QPUi')
get_file('https://ufrpebr-my.sharepoint.com/:u:/g/personal/joseantonio_santos_ufrpe_br/ESigtraTg-xLjUAwHuEzF0sB-CkOhU75tNSZcomdbuvpxg?e=AcMaXi')

Archive:  /content/data/base_20230428_douglas-bill_corpus-parts.zip
  inflating: /content/data/base_20230428_douglas-bill_corpus-part0.csv  
  inflating: /content/data/base_20230428_douglas-bill_corpus-part1.csv  
  inflating: /content/data/base_20230428_douglas-bill_corpus-part2.csv  
  inflating: /content/data/base_20230428_douglas-bill_corpus-part3.csv  
  inflating: /content/data/base_20230428_douglas-bill_corpus-part4.csv  
Unziped and Saved base_20230428_douglas-bill_corpus-parts.zip
Saved pares-arvores-proposicao-filter.csv
Archive:  /content/data/dados-conle-anonimizado-job_request-parts.zip
  inflating: /content/data/dados-conle-anonimizado-job_request-part0.csv  
  inflating: /content/data/dados-conle-anonimizado-job_request-part1.csv  
  inflating: /content/data/dados-conle-anonimizado-job_request-part2.csv  
  inflating: /content/data/dados-conle-anonimizado-job_request-part3.csv  
  inflating: /content/data/dados-conle-anonimizado-job_request-part4.csv  
Unziped and Saved 

# Evaluate


In [ ]:
def evaluate(df, arr_assunto, top_k, all_preprocessing, column_data_bm25,column_data_model, model_name):
    data = df[column_data_bm25].to_numpy()
    if all_preprocessing:
        tokenized_corpus3 = [preprocess3(doc) for doc in data]                      #Stopwords + Acentuação + Stemming
        bigram_corpus_stem = [preprocess_ngram_stem(doc, 2) for doc in data]        #Stopwords + Acentuação + Stemming + Bigram
        bm25L_bigram_stem = BM25L(bigram_corpus_stem)           #Stopwords + Acentuação + Stemming + Bigram             #Tokenizer
    else:
        tokenized_corpus3 = [preprocess0(doc) for doc in data]
    bm25L_tokenstem = BM25L(tokenized_corpus3)              #Stopwords + Acentuação + Stemming

    y,X = arr_assunto[:,0],arr_assunto[:,1]
    y = [i.strip() for i in y]
    model = SentenceTransformer(model_name)
    model.max_seq_length=512

    corpus_embeddings = model.encode(df[column_data_model].to_numpy(), convert_to_tensor=True)

    '''
    Obter o resultado do BM25 para cada query
    '''
    #Listas com os n documentos mais relevantes para cada versao do BM25
    labels_nsL = list()             #Stopwords + Acentuação + Stemming + Unigram + Bigram (L)
    l_v = list()
    for l,x in zip((y),X):
        if all_preprocessing:
            tokenized_query3 = preprocess3(x)                   #Stopwords + Acentuação + Stemming
        else:
            tokenized_query3 = preprocess0(x)                   #Tokenizer
        query_embedding = model.encode(x, convert_to_tensor=True)
        scores_stem_l = list()
        scores = bm25L_tokenstem.get_partial_score(tokenized_query3, data)
        scores_normalized = (scores - np.min(scores)) / (np.max(scores) - np.min(scores))


        model_score = evaluate_model(query_embedding=query_embedding, corpus_embeddings=corpus_embeddings)
        model_score_normalized = (model_score - np.min(model_score)) / (np.max(model_score) - np.min(model_score))
        if all_preprocessing:
            # final_score = scores_normalized + scores_bi_normalized + model_score_normalized
            final_score = scores_normalized + model_score_normalized
        else:
            final_score = scores_normalized + model_score_normalized
        scores_stem_l.append(final_score)
        top_n_stem_l = bm25L_tokenstem.get_top_n_ngram(scores_stem_l, data, n=top_k)

        labelsnsL = [get_name(df,d,column_data_bm25) for d in top_n_stem_l]              #L
        #Adicionar os resultados da query as listas
        labels_nsL.append(labelsnsL)    #L

        #Adicionar resposta esperada a lista
        l_v.append(l)

    res_nsL = list()                #Stopwords + Acentuação + Stemming + Unigram + Bigram (L)
    for k in range(1, top_k+1):
        resultsnsL = list()         #Stopwords + Acentuação + Stemming + Unigram + Bigram (L)
        for i in range(len(l_v)):
            if (l_v[i] in labels_nsL[i][:k]):
                resultsnsL.append(1)
            else:
                resultsnsL.append(0)
        res_nsL.append(sum(resultsnsL)/len(resultsnsL))

    return res_nsL[-1]

In [ ]:
'''
Verificar se a solicitação correta esta entre os k primeiro documentos retornados (de 1 ate k=20)
'''
# top_k = 20
# column_data_bm25 = 'txtInteiroTeorLimpo'
# column_data_model = 'txtInteiroTeorLimpo'
# version = '/content/drive/MyDrive/Sentence Model Tuned/txtInteiroTeor_sbert'
runs = 5
recalls_processed = []

for i in tqdm(range(0, runs)):
    element = f'{column_data_bm25}_{i}'
    df = pd.read_csv(f"/content/data/base_20230428_douglas-bill_corpus-part{i}.csv")
    df_assunto = pd.read_csv(f"/content/data/dados-conle-anonimizado-job_request-part{i}.csv", encoding="utf-8").to_numpy()
    recalls_processed.append(evaluate(df,df_assunto,top_k, True,column_data_bm25,column_data_model,version))
print(f'BM25L preprocessed {recalls_processed} --> Média: {round(np.average(recalls_processed)*100,2)}. Desvio Padrão: {np.std(recalls_processed)}')


100%|██████████| 5/5 [2:01:46<00:00, 1461.32s/it]

BM25L preprocessed [0.6888888888888889, 0.8648648648648649, 0.8656716417910447, 0.9622641509433962, 0.8571428571428571] --> Média: 84.78. Desvio Padrão: 0.08837722742970304


In [ ]:
'''
Verificar se a solicitação correta esta entre os k primeiro documentos retornados (de 1 ate 100)
'''
top_k = 20
column_data_bm25 = 'txtInteiroTeor'
column_data_model = 'txtInteiroTeor'
version = '/content/drive/MyDrive/Sentence Model Tuned/txtInteiroTeor_sbert'
runs = 5
recalls_processed = []

for i in tqdm(range(0, runs)):
    element = f'{column_data_bm25}_{i}'
    df = pd.read_csv(f"/content/data/base_20230428_douglas-bill_corpus-part{i}.csv")
    df_assunto = pd.read_csv(f"/content/data/dados-conle-anonimizado-job_request-part{i}.csv", encoding="utf-8").to_numpy()
    recalls_processed.append(evaluate(df,df_assunto,top_k, True,column_data_bm25,column_data_model,version))

print(f'BM25L preprocessed {recalls_processed} --> Média: {round(np.average(recalls_processed)*100,2)}. Desvio Padrão: {np.std(recalls_processed)}')


100%|██████████| 5/5 [1:42:09<00:00, 1225.97s/it]

BM25L preprocessed [0.6888888888888889, 0.8648648648648649, 0.8656716417910447, 0.9622641509433962, 0.8571428571428571] --> Média: 84.78. Desvio Padrão: 0.08837722742970304
